In [48]:
%matplotlib

In [49]:
# Work through notebook

In [345]:
import lib.ga_utils as g
import lib.auth as a
import lib.analytics as Analytics
from datetime import datetime, date, timedelta
import calendar
import time
import importlib
from pprint import pprint
import pandas as pd
import numpy as np

In [108]:
importlib.reload(a)
auth = a.Auth('jupyter')
importlib.reload(Analytics)
analytics = auth.get_analytics()

In [320]:
prop = analytics.get_prop('fullstack.io', 'ng-book.com')
profile = prop.profiles[0]

In [325]:
[m.id for m in profile.core.metrics]
[m.id for m in profile.core.dimensions]

['ga:userType',
 'ga:sessionCount',
 'ga:daysSinceLastSession',
 'ga:userDefinedValue',
 'ga:sessionDurationBucket',
 'ga:referralPath',
 'ga:fullReferrer',
 'ga:campaign',
 'ga:source',
 'ga:medium',
 'ga:sourceMedium',
 'ga:keyword',
 'ga:adContent',
 'ga:socialNetwork',
 'ga:hasSocialSourceReferral',
 'ga:adGroup',
 'ga:adSlot',
 'ga:adDistributionNetwork',
 'ga:adMatchType',
 'ga:adKeywordMatchType',
 'ga:adMatchedQuery',
 'ga:adPlacementDomain',
 'ga:adPlacementUrl',
 'ga:adFormat',
 'ga:adTargetingType',
 'ga:adTargetingOption',
 'ga:adDisplayUrl',
 'ga:adDestinationUrl',
 'ga:adwordsCustomerID',
 'ga:adwordsCampaignID',
 'ga:adwordsAdGroupID',
 'ga:adwordsCreativeID',
 'ga:adwordsCriteriaID',
 'ga:adQueryWordCount',
 'ga:goalCompletionLocation',
 'ga:goalPreviousStep1',
 'ga:goalPreviousStep2',
 'ga:goalPreviousStep3',
 'ga:browser',
 'ga:browserVersion',
 'ga:operatingSystem',
 'ga:operatingSystemVersion',
 'ga:mobileDeviceBranding',
 'ga:mobileDeviceModel',
 'ga:mobileInputSel

In [246]:
# prev_month = date.today().replace(day=1) - timedelta(days=1)
# query = profile.core.query \
#     .range(prev_month, months=1) \
#     .metrics('ga:sessionsPerUser', 'pageviews') \
#     .dimensions('date', 'pagePath', 'country', 'browser') \
#     .sort('date', descending=True)

# report = query.get()
# pprint(report.raw)

In [430]:
def testfn(**kwargs):
    args = kwargs.get('list', ('hello', 'world'))
    print(args)
    
testfn(list=('hello'))

hello


In [619]:
def days_ago(n):
    return datetime.now() - timedelta(days=n)

def range_query(**kwargs):
    run      = kwargs.get('run', True)
    start_at = kwargs.get('start_at', days_ago(1))
    end_at   = kwargs.get('end_at', start_at)
    
    default_metrics = ('sessions', 'pageviews', 'session duration', 'bounces')
    default_dimensions = ('date', 'source', 'medium')
    
    metrics  = kwargs.get('metrics', default_metrics)
    dimensions = kwargs.get('dimensions', default_dimensions)
    sortBy     = kwargs.get('sortBy', False)
    
    query = profile.core.query \
                .range(start_at, end_at) \
                .metrics(*metrics) \
                .dimensions(*dimensions)
    
    if sortBy:
        query = query.sort(sortBy, descending=True)
    
    return query.get() if run else query

def revenue_generating_campaigns(**kwargs):
    opts = dict(metrics=('sessions', 'ga:transactionRevenue', 'transactions', 'unique purchases'), \
                dimensions=('source', 'medium'), \
                sortBy='sessions', \
                **kwargs)
    return range_query(**opts)

def top_content(**kwargs):
    opts = dict(metrics=('pageviews', \
                         'unique pageviews',\
                         'time on page',\
                         'bounces', 'entrances','exits'),\
                dimensions=('ga:pagePath', 'ga:fullReferrer', 'ga:referralPath'),\
                sortBy='pageviews', **kwargs)
    return range_query(**opts)

def get_last_n_days(n, **kwargs):
    interval = kwargs.get('interval', 'days')
    ago = datetime.now() - timedelta(days=n)
    return pd.date_range(ago, periods=n, freq='D')

def data_for_last_n_days(n, **kwargs):
    data = {}
    func = kwargs.get('func', range_query)
    for date in get_last_n_days(n):
        opts = dict(start_at=date, **kwargs)
        data[str(date)] = func(**opts)
    return data

def get_pd(name, **kwargs):
    n = kwargs.get('days', 10)
    
    data = data_for_last_n_days(n, **kwargs)
    df = pd.concat([data[d].as_dataframe() for d in data.keys()])
    df.to_pickle('../data/' + name + '.pkl')
    return df

In [618]:
rev = get_pd('revenue_generating', \
                        func=revenue_generating_campaigns,\
                        days=90)
cont = get_pd('content',\
             func=top_content,\
             days=90)

range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: ('source', 'medium')
range query: (

In [627]:
# df.groupby('unique_purchases').sum().sort_values(['transactions'], ascending=False)
cont.groupby('full_referrer').sum()\
    .sort_values(['pageviews'], ascending=False)
# cont.groupby(['source'])\
#     .sum()\
#     .sort_values(['sessions'], ascending=False)

,pageviews,unique_pageviews,time_on_page,bounces,entrances,exits
full_referrer,,,,,,
google,214317,178778,9057530.0,127299,159557,159557
(direct),62510,43555,2024546.0,17007,30112,30112
fullstackreact.com/,7499,4096,153170.0,2,1788,1788
whatpixel.com/learn-angular2-from-scratch/,5069,4080,181267.0,2810,3742,3742
javascriptweekly,5011,4182,221399.0,3177,3945,3945
angular.io/,3353,2962,137818.0,2371,2747,2747
NG-Newsletter_160,3084,2573,157493.0,1688,2270,2270
medium.com/russian/frontend-dev-%D1%85%D0%BE%D1%80%D0%BE%D1%88%D0%B8%D0%B9-%D0%BF%D0%BB%D0%BE%D1%85%D0%BE%D0%B9-%D0%B7%D0%BB%D0%BE%D0%B9-%D1%87%D0%B0%D1%81%D1%82%D1%8C-2-%D0%BF%D0%BB%D0%BE%D1%85%D0%BE%D0%B9-%D0%BF%D1%83%D1%82%D1%8C-aa99f3ce376,2872,2628,109554.0,2195,2482,2482
codementor.io/angularjs/tutorial/angularjs-interview-questions-sample-answers,2714,2526,150007.0,2010,2297,2297


In [411]:
# y = Mx
# grouped by sessions x sessions with transactions
# Every page visited, how long did they visit
# Turn pages into a bool feature
# For every session with a transaction, increment a counter for that feature
# which has weight
#
# Normalize on each feature

In [315]:
## Read here to save on api calls
df = pd.read_pickle('analytics.pkl')

In [407]:
byUserType     = df.groupby(['user_type'])
viewsByBrowser = df.groupby(['browser'])
viewsByCountry = df.groupby(['country'])

df
# viewsByBrowser.aggregate(np.sum)
# d['viewsByBrowser'] = d.browser.apply(viewsByBrowser.get_value)

# d.sort_values(by='viewsByBrowser', ascending=False)
# d.sort_values(by='pageviews',ascending=False)
# byBrowser[['pageviews']].transform(sum).sort_values(by='pageviews')

,date,page_path,country,browser,user_type,sessions_per_user,pageviews,total_value,transaction_revenue
0,2016-09-19,/,(not set),Chrome,New Visitor,1.000000,1,0.0,0.0
1,2016-09-19,/,(not set),Opera,New Visitor,0.000000,1,0.0,0.0
2,2016-09-19,/,(not set),Opera Mini,New Visitor,1.000000,1,0.0,0.0
3,2016-09-19,/,Argentina,Chrome,New Visitor,1.000000,4,0.0,0.0
4,2016-09-19,/,Argentina,Chrome,Returning Visitor,1.000000,1,0.0,0.0
5,2016-09-19,/,Armenia,Chrome,New Visitor,1.000000,1,0.0,0.0
6,2016-09-19,/,Australia,Chrome,New Visitor,1.000000,2,0.0,0.0
7,2016-09-19,/,Australia,Chrome,Returning Visitor,0.000000,3,0.0,0.0
8,2016-09-19,/,Australia,Internet Explorer,New Visitor,1.000000,1,0.0,0.0
9,2016-09-19,/,Austria,Chrome,New Visitor,0.000000,1,0.0,0.0


In [33]:
df['country'].value_counts(ascending=False)

United States        58
India                46
China                24
Ukraine              21
Canada               20
Germany              19
France               19
Italy                18
United Kingdom       16
Russia               15
Vietnam              15
Poland               14
Brazil               14
Spain                14
Turkey               12
Netherlands          11
Japan                11
Singapore            11
Belarus              11
Romania              10
(not set)            10
Thailand             10
Indonesia             9
Serbia                9
Australia             9
Hong Kong             9
Mexico                8
Colombia              8
Argentina             8
New Zealand           8
                     ..
Cambodia              2
Luxembourg            2
Iceland               2
Albania               2
Syria                 2
Cyprus                2
Latvia                2
Maldives              1
Ethiopia              1
Uganda                1
Sierra Leone    